In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

import matplotlib.pyplot as plt

import syft as sy
hook=sy.TorchHook(torch)

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/root/anaconda3/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0.so'


### Creating two clients and fitting a linear regression made with a neural network

In [2]:
#torch.rand => uniform distribution

dim=100

X1=torch.rand(dim, 1) 
X2=torch.rand(dim,1)

a_1,b_1=5,-1
a_2,b_2=a_1,b_1#a_2,b_2=3,2
y1=a_1*X1+b_1+0.5*torch.randn(dim,1)
y2=a_2*X2+b_2+0.5*torch.randn(dim,1)

### FedAvg for the two clients created above

In [3]:
#Create the server and the clients
server = sy.VirtualWorker(hook, id="server")
C1 = sy.VirtualWorker(hook, id="C1")
C2 = sy.VirtualWorker(hook, id="C2")

#Send the features and the labels to its associated client
C1_x = X1.send(C1)
C1_y = y1.send(C1)

C2_x = X2.send(C2)
C2_y = y2.send(C2)

In [4]:
#torch model used for the inference
model = nn.Linear(1,1)

loss_f=nn.MSELoss()
iterations=100
epochs=5

print(model.weight.data.numpy(),model.bias.data.numpy())

for i in range(iterations):
    C1_model = model.copy().send(C1)
    C2_model = model.copy().send(C2)
    
    C1_opt = optim.SGD(params=C1_model.parameters(),lr=0.1)
    C2_opt = optim.SGD(params=C2_model.parameters(),lr=0.1)
    
    for j in range(epochs):
        
        # Train Bob's Model
        C1_opt.zero_grad()
        C1_pred = C1_model(C1_x)
        C1_loss = loss_f(C1_pred,C1_y)
        C1_loss.backward()

        C1_opt.step()
        C1_loss = C1_loss.get().data

        # Train Alice's Model
        C2_opt.zero_grad()
        C2_pred = C2_model(C2_x)
        C2_loss = loss_f(C2_pred,C2_y)
        C2_loss.backward()

        C2_opt.step()
        C2_loss = C2_loss.get().data

        print("C1:" + str(C1_loss.detach().numpy()) + " C2:" + str(C2_loss.detach().numpy()))
        
    C1_model.move(server)
    C2_model.move(server)

    with torch.no_grad():
        model.weight.set_(((C1_model.weight.data + C2_model.weight.data) / 2).get())
        model.bias.set_(((C1_model.bias.data + C2_model.bias.data) / 2).get())
    
    print(model.weight.data.numpy(),model.bias.data.numpy())

[[-0.9454144]] [-0.46929252]
C1:8.133355 C2:9.146053
C1:5.564692 C2:5.814426
C1:4.0856237 C2:3.9678311
C1:3.2260816 C2:2.9375184
C1:2.7189908 C2:2.3560574
[[0.15516534]] [0.92440367]
C1:2.2883027 C2:2.170424
C1:2.1433904 C2:1.9100518
C1:2.0438364 C2:1.7525569
C1:1.9704806 C2:1.6519275
C1:1.9124267 C2:1.5828762
[[0.6139738]] [1.1441351]
C1:1.8543319 C2:1.557515
C1:1.8161602 C2:1.5024817
C1:1.7792566 C2:1.4594873
C1:1.743452 C2:1.4234107
C1:1.708641 C2:1.391431
[[0.91053796]] [1.0956271]
C1:1.684623 C2:1.3659014
C1:1.6501514 C2:1.3331182
C1:1.6170816 C2:1.3038726
C1:1.5851438 C2:1.2768377
C1:1.5541763 C2:1.2512783
[[1.1577653]] [0.9903775]
C1:1.5364808 C2:1.2259287
C1:1.504782 C2:1.1993634
C1:1.4748105 C2:1.174886
C1:1.446118 C2:1.1517909
C1:1.4184425 C2:1.1296846
[[1.3827534]] [0.87750196]
C1:1.4033899 C2:1.1062818
C1:1.3750687 C2:1.0834441
C1:1.3483623 C2:1.0622084
C1:1.3228377 C2:1.0420594
C1:1.2982419 C2:1.0227093
[[1.5924852]] [0.7681508]
C1:1.2850178 C2:1.0015289
C1:1.2599328 C2:0.

[[4.713929]] [-0.8794956]
C1:0.35752797 C2:0.24579349
C1:0.35697812 C2:0.24537505
C1:0.35659787 C2:0.245145
C1:0.35631573 C2:0.2450184
C1:0.35609093 C2:0.24494866
[[4.7227707]] [-0.88416374]
C1:0.3571414 C2:0.24579495
C1:0.35659444 C2:0.24536936
C1:0.35621998 C2:0.2451355
C1:0.35594508 C2:0.24500696
C1:0.3557283 C2:0.24493635
[[4.73108]] [-0.888551]
C1:0.35678965 C2:0.24580736
C1:0.3562451 C2:0.24537472
C1:0.3558757 C2:0.245137
C1:0.3556073 C2:0.24500641
C1:0.35539773 C2:0.24493468
[[4.7388897]] [-0.8926743]
C1:0.3564692 C2:0.24582876
C1:0.3559266 C2:0.24538922
C1:0.35556173 C2:0.24514763
C1:0.3552992 C2:0.24501488
C1:0.35509622 C2:0.24494185
[[4.7462296]] [-0.89654964]
C1:0.35617703 C2:0.24585748
C1:0.355636 C2:0.24541119
C1:0.35527515 C2:0.24516577
C1:0.3550179 C2:0.24503073
C1:0.35482088 C2:0.24495633
[[4.753128]] [-0.9001919]
C1:0.35591042 C2:0.2458921
C1:0.35537064 C2:0.24543923
C1:0.35501328 C2:0.24519
C1:0.35476083 C2:0.24505264
C1:0.3545692 C2:0.24497677
[[4.759611]] [-0.903615

### FedAvg generalization for any neural network

In [5]:
def FedAvg(w0,B=1,E=2,eta=10**-2,C=1):
    #initialization of the model's weights
    #B local minibatch size
    #E number of local epochs
    #eta learning rate
    #C percentage of considered clients at a given step

SyntaxError: unexpected EOF while parsing (<ipython-input-5-f5f8fa519d77>, line 6)

### FedAvg generalization to keep the gradient and the model parameters at each step